# Data

## Loading the Data

## Data Cleansing

## Data Preprocessing

## Visualizing the Data

## Splitting the Datasets 

# Machine Learning Modelling

## Creating the Model

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

class FeedForwardNN(nn.Module): 
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()  # Fixed super call

        self.linear_stack = nn.Sequential(
            nn.Linear(input_shape, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, hidden_units),
            nn.ReLU(),
            nn.Linear(hidden_units, output_shape),
        )

    def forward(self, x: torch.Tensor):
        return self.linear_stack(x)

model = FeedForwardNN(input_shape=3, hidden_units=128, output_shape=1).to(device)


## Fitting the Model with Training Data

## Verify if Fitting is performed correctly

# Perform Predictions with the Model

# Evaluate the Performance of the Model

# Conclusion and Further Discussion